In [56]:
# Dependencies
import json
import pandas as pd
from datetime import datetime
import numpy as np

In [57]:
#read the data from saved files so we don't overuse our API
with open('./Resources/weather_2015_to_2017_json.txt') as json_file:
    weather_response = json.load(json_file)
    
with open('./Resources/crime_2015_json.txt') as json_file:
    response_2015 = json.load(json_file)
    
with open('./Resources/crime_2016_json.txt') as json_file:
    response_2016 = json.load(json_file)
    
with open('./Resources/crime_2017_json.txt') as json_file:
    response_2017 = json.load(json_file)    

In [58]:
#loop through weather data, convert data
#get the weather data we are interested in analysing
date = []
temp = []
# Using loop
# Get values of particular key in list of dictionaries 
for weather in weather_response:
    try:
        date.append(datetime.fromtimestamp(weather['dt']))
        temp.append(round(weather['main']['temp']))       
    except:
        print('exception')
        pass

In [59]:
# create a data frame from the weather data
weather_dict = {
    'date': date,
    'temp': temp
}
weather_data = pd.DataFrame(weather_dict)
weather_data.head(10)

,date,temp
0,2014-12-31 19:00:00,18
1,2014-12-31 20:00:00,19
2,2014-12-31 21:00:00,19
3,2014-12-31 22:00:00,19
4,2014-12-31 23:00:00,19
5,2015-01-01 00:00:00,19
6,2015-01-01 01:00:00,18
7,2015-01-01 02:00:00,18
8,2015-01-01 03:00:00,18
9,2015-01-01 04:00:00,17


In [60]:
# we have weather data from 2014, this is because the data is captured in GMT, 
# GMT is 6 hours ahead of CST, the time zone for Chicago

In [61]:
received_weather_records=weather_data['temp'].count()

In [62]:
# there are 365 days in 2015
# there are 366 days in 2016
# there are 365 days in 2017
# there are weather measurements every hours, resulting in 24 weather records per day.
# The expected number of weather records is
# (365*24) + (366*24) + (365*24) = 26304
expected_weather_records = (365*24) + (366*24) + (365*24)
print(f'weather records we expected {expected_weather_records}')
print(f'weather records we received {received_weather_records}')

weather records we expected 26304
weather records we received 26307


In [63]:
# explain why we do nothave the expected number of weather records
# add year so we can find weather records outside out time frame
# add the truncated date so we can find days that do not have 24 weather records
# source reference to extract year from pandas
# https://www.interviewqs.com/ddi_code_snippets/extract_month_year_pandas
weather_data_year = weather_data
weather_data_year['date_trunc'] = weather_data_year['date'].dt.floor('D')
weather_data_year['year'] = pd.DatetimeIndex(weather_data_year['date']).year

In [64]:
weather_data_year

,date,temp,date_trunc,year
0,2014-12-31 19:00:00,18,2014-12-31,2014
1,2014-12-31 20:00:00,19,2014-12-31,2014
2,2014-12-31 21:00:00,19,2014-12-31,2014
3,2014-12-31 22:00:00,19,2014-12-31,2014
4,2014-12-31 23:00:00,19,2014-12-31,2014
...,...,...,...,...
26302,2017-12-31 14:00:00,12,2017-12-31,2017
26303,2017-12-31 15:00:00,14,2017-12-31,2017
26304,2017-12-31 16:00:00,14,2017-12-31,2017
26305,2017-12-31 17:00:00,13,2017-12-31,2017


In [65]:
# get the count of records for years other than 2015-2017
odd_days = (weather_data_year['date'][(weather_data_year['year'] != 2015) & 
                                               (weather_data_year['year'] != 2016) &
                                               (weather_data_year['year'] != 2017)]
           )
records_outside_range = len(odd_days)       
print(f'extra records outside our range {records_outside_range}' )

extra records outside our range 5


In [66]:
# get the weather row count by day to see what weather we have
# source reference
# filter based on multiple values 
#https://www.interviewqs.com/ddi_code_snippets/rows_cols_python
years = [2015,2016,2017]
weather_data_day = weather_data_year[['date','date_trunc']][weather_data_year['year'].isin(years)]
                     
grouped_weather_data_count = weather_data_day.groupby(['date_trunc']).count()

# days where we don't have 24 weather records
odd_days = grouped_weather_data_count['date'][grouped_weather_data_count['date'] != 24 ]

# convert to a dataframe to easier use
odd_days = pd.DataFrame(odd_days)

# determine how many weather data records we are missing
odd_days['delta'] = odd_days['date'] - 24
records_delta = odd_days['delta'].sum()
print(f'we expected {expected_weather_records} weather records')
print(f'we received {received_weather_records} weather records')
print(f'We were short weather records for the date range 2015 - 2017 by {records_delta}')
print(f'we were over weather records outside our date range 2015-2017 by {records_outside_range}')
print(f'This explains the delta between what we received and what we expected to recieve')

we expected 26304 weather records
we received 26307 weather records
We were short weather records for the date range 2015 - 2017 by -2
we were over weather records outside our date range 2015-2017 by 5
This explains the delta between what we received and what we expected to recieve


In [67]:
# sort the data and remove duplicates
#source reference
#https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/
weather_data.sort_values('date', inplace = True) 


In [68]:
#remove missing values from the dataframe
weather_data.drop_duplicates(subset ='date', keep = False, inplace = True) 

In [69]:
# write cleaned weather data to a csv
weather_data.to_csv('./Resources/weather_2015_to_2017.csv',index=False)

In [70]:
# load crime data into dataframes
crime_2015_df=pd.DataFrame(response_2015)
crime_2016_df=pd.DataFrame(response_2016)
crime_2017_df=pd.DataFrame(response_2017)

In [71]:
#concatenate the dataframes
#source reference
#https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
frames = [crime_2015_df,crime_2016_df,crime_2017_df]
crime_df = pd.concat(frames,sort=True)
print(f' rows after concat {len(crime_df.index)}')

 rows after concat 802022


In [72]:
crime_df.head()

,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_d3ds_rm58,:@computed_region_d9mm_jgwp,:@computed_region_rpca_8um6,:@computed_region_vrxf_vc4k,arrest,beat,...,latitude,location,location_description,longitude,primary_type,updated_on,ward,x_coordinate,y_coordinate,year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1422,...,NaN,NaN,RESIDENCE PORCH/HALLWAY,NaN,THEFT,2018-02-09T15:44:29.000,26,NaN,NaN,2015
1,32,21554,17,511,229,20,59,66,False,0621,...,41.757366519,"{'latitude': '41.757366519', 'human_address': ...",STREET,-87.642992854,CRIMINAL DAMAGE,2018-02-10T15:50:01.000,17,1172605,1854931,2015
2,35,21202,9,431,224,19,25,41,False,0411,...,41.751270452,"{'latitude': '41.751270452', 'human_address': ...",STREET,-87.585822373,BATTERY,2018-02-10T15:50:01.000,8,1188223,1852840,2015
3,27,22528,42,359,35,11,12,20,False,2411,...,42.016804165,"{'latitude': '42.016804165', 'human_address': ...",APARTMENT,-87.690708662,THEFT,2018-02-10T15:50:01.000,50,1158878,1949369,2015
4,25,21186,38,691,12,5,53,57,True,1923,...,41.949837364,"{'latitude': '41.949837364', 'human_address': ...",SIDEWALK,-87.658635101,BATTERY,2018-02-10T15:50:01.000,44,1167786,1925033,2015


In [73]:
#verify we have all the crime data
crime_2015_count = 264306
crime_2016_count = 269258
crime_2017_count = 268458
expected_crime_rows = crime_2015_count + crime_2016_count + crime_2017_count
received_crime_rows = len(crime_2015_df.index) + len(crime_2016_df.index) + len(crime_2017_df.index) 
print(f'crime rows we expected to receive {expected_crime_rows}')
print(f'crime rows we actually received {received_crime_rows}')

crime rows we expected to receive 802022
crime rows we actually received 802022


In [74]:
#Reduce to columns we need for temp analysis
#Remove missing values
sub_crime_df = crime_df[['date','primary_type']]
sub_crime_df.set_index('date')
sub_crime_df = sub_crime_df.dropna(axis=0, how='any', thresh=None, subset=None)
print(f' crime rows after dropna {len(crime_df.index)}')

 crime rows after dropna 802022


In [75]:
#get the crime datetime at the hour so we can join to the hourly historical weather data
#https://stackoverflow.com/questions/28773342/truncate-timestamp-column-to-hour-precision-in-pandas-dataframe
sub_crime_df['date'] = pd.to_datetime(sub_crime_df['date'])
sub_crime_df['date'] = sub_crime_df['date'].dt.floor('H')

In [76]:
# write cleaned crime data to a csv
sub_crime_df.to_csv('./Resources/crime_2015_to_2017.csv',index=False)

In [77]:
# Merge the weather & crime dataframes using an left join on date
#source reference
#https://stackoverflow.com/questions/26283641/pandas-dataframe-too-long-after-merge
merge_table = pd.merge(sub_crime_df, weather_data, left_on='date',right_on='date',how='left')
print(f'expected rows after merge of crime and weather {len(crime_df.index)}')
print(f'rows after merge of crime and weather {len(merge_table.index)}')

expected rows after merge of crime and weather 802022
rows after merge of crime and weather 802022


In [78]:
# Remove missing values
merge_table = merge_table.dropna(axis=0, how='any', thresh=None, subset=None)
print(f'merged rows after dropna {len(merge_table.index)}')

merged rows after dropna 801551


In [79]:
# we lost data due to GMT weather data
print(f'crime rows lost due to missing weather data due to 6 hour GMT time difference { len(crime_df.index) -len(merge_table.index) }')

crime rows lost due to missing weather data due to 6 hour GMT time difference 471


In [80]:
merge_table.head()

,date,primary_type,temp,date_trunc,year
0,2015-12-31 23:00:00,THEFT,24.0,2015-12-31,2015.0
1,2015-12-31 23:00:00,CRIMINAL DAMAGE,24.0,2015-12-31,2015.0
2,2015-12-31 23:00:00,BATTERY,24.0,2015-12-31,2015.0
3,2015-12-31 23:00:00,THEFT,24.0,2015-12-31,2015.0
4,2015-12-31 23:00:00,BATTERY,24.0,2015-12-31,2015.0


In [81]:
merge_table.dtypes

date            datetime64[ns]
primary_type            object
temp                   float64
date_trunc      datetime64[ns]
year                   float64
dtype: object

In [82]:
# write merged crime and weather data to a csv
merge_table.to_csv('./Resources/merged_crime_and_weather_data.csv',index=False)